In [7]:
import numpy as np
import networkx as nx
import graph_generator as gg
import graph_visualiser as gv
import importlib

In [8]:
importlib.reload(gg)
importlib.reload(gv)

<module 'graph_visualiser' from 'C:\\Users\\michn\\PycharmProjects\\pthong\\MOWNiT-Lab\\lab2\\graph_visualiser.py'>

In [9]:
def kirchoff(g, s, t, E):
    def add_sum_component(v1, v2):
        edge = g[v1][v2]['edge']
        if edge.voltage > 0:
            if v1 < v2:
                B[eq_no] -= edge.voltage
            else:
                B[eq_no] += edge.voltage
        else:
            if v1 < v2:
                A[eq_no][edge.index] += edge.resistance
            else:
                A[eq_no][edge.index] -= edge.resistance

    n = g.number_of_nodes()
    m = g.number_of_edges()

    cycle_basis = nx.cycle_basis(g)
    no_of_eq = n + len(cycle_basis)
    A = np.zeros((no_of_eq, m))
    B = np.zeros(no_of_eq)

    eq_no = 0
    for cycle in cycle_basis:
        for i in range(1, len(cycle)):
            add_sum_component(cycle[i-1], cycle[i])
        add_sum_component(cycle[-1], cycle[0])
        eq_no += 1

    for v in range(n):
        for neighbor in g.neighbors(v):
            edge = g[v][neighbor]['edge']
            if v < neighbor:
                A[eq_no][edge.index] += 1
            else:
                A[eq_no][edge.index] -= 1
        eq_no += 1

    transposed = A.T
    ans = np.linalg.solve(np.matmul(transposed, A), np.matmul(transposed, B))
    for a, b in g.edges:
        edge = g[a][b]['edge']
        edge.current = ans[edge.index]

In [10]:
def nodal_potentials(g, s, t, E):
    n = g.number_of_nodes()
    A = np.zeros((n, n))
    B = np.zeros(n)
    A[s][s] = 1
    B[s] = E
    A[t][t] = 1
    B[t] = 0

    for v in range(n):
        if v != s and v != t:
            for neighbor in g.neighbors(v):
                edge = g[v][neighbor]['edge']
                A[v][v] += 1/edge.resistance
                A[v][neighbor] -= 1/edge.resistance

    potentials = np.linalg.solve(A, B)
    for a, b in g.edges:
        edge = g[a][b]['edge']
        if a < b:
            edge.current = (potentials[a] - potentials[b]) / edge.resistance
        else:
            edge.current = (potentials[b] - potentials[a]) / edge.resistance

In [11]:
def check_currents(g, s, t):
    eps = pow(10, -12)
    for v in range(g.number_of_nodes()):
        if v != s and v != t:
            current_sum = 0
            for neighbor in g.neighbors(v):
                if v < neighbor:
                    current_sum += g[v][neighbor]['edge'].current
                else:
                    current_sum -= g[v][neighbor]['edge'].current
            if abs(current_sum) > eps:
                return False
    return True